In [1]:
import sys
import os

# Add the project root directory to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)
print(f"Project root added to sys.path: {project_root}")

Project root added to sys.path: c:\Users\sylla\Desktop\MLOPS\Project1_with_deployment


In [2]:
import sys
import os

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/lassoinadame2018/Project1_Data_Science.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "lassoinadame2018"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "f87541f1d8f0408eeabe6b26ca2059a57b8778cd"

In [3]:
%pwd

'c:\\Users\\sylla\\Desktop\\MLOPS\\Project1_with_deployment\\Project1_Data_Science\\research'

In [4]:
os.chdir("../")
%pwd

'c:\\Users\\sylla\\Desktop\\MLOPS\\Project1_with_deployment\\Project1_Data_Science'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    TARGET_COLUMN : str
    mlflow_uri: str
    all_params : dict

### Configuation Manager

In [6]:
from src.Project1_Data_Science.constants import *
from src.Project1_Data_Science.utils.common import *
from src.Project1_Data_Science.entity import *


In [7]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath: str = CONFIG_FILE_PATH,
                 params_filepath: str = PARAMS_FILE_PATH,
                 schema_filepath: str = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            metric_file_name= config.metric_file_name,

            TARGET_COLUMN= schema.name,
            mlflow_uri= "https://dagshub.com/lassoinadame2018/Project1_Data_Science.mlflow",
            all_params= params
        )
        return model_evaluation_config

In [8]:
import os
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
import joblib

In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, y_pred):
        rmse = np.sqrt(mean_squared_error(actual, y_pred))
        mae = mean_absolute_error(actual, y_pred)
        r2 = r2_score(actual, y_pred)
        return rmse, mae, r2

    # define model evluation method
    def log_into_mlflow(self):
        # Load test dataset
        test_data = pd.read_csv(self.config.test_data_path)
        # Load model
        model = joblib.load(self.config.model_path)

        # Extract features and target variable
        X_test = test_data.drop(columns=[self.config.TARGET_COLUMN])
        y_test = test_data[[self.config.TARGET_COLUMN]]

        # set mlflow tracking uri
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_score = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            # Predict using the model
            y_pred = model.predict(X_test)

            # Calculate metrics
            (rmse, mae, r2) = self.eval_metrics(y_test, y_pred)
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path= Path(self.config.metric_file_name), data=scores)

            # Log metrics to MLflow
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            if tracking_uri_type_score != "file":
                # registry the model
                #There are other way ton use model registry, which depends on the use case
                
                # 1. Register the model
                mlflow.sklearn.log_model(model, 'model',registered_model_name ="ElasticNetModel")

            else :
                # 2. Log the model
                mlflow.sklearn.log_model(model, "model")




In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

Lecture de YAML depuis C:\Users\sylla\Desktop\MLOPS\Project1_with_deployment\Project1_Data_Science\config\config.yaml
Lecture de YAML depuis C:\Users\sylla\Desktop\MLOPS\Project1_with_deployment\Project1_Data_Science\params.yaml
Lecture de YAML depuis C:\Users\sylla\Desktop\MLOPS\Project1_with_deployment\Project1_Data_Science\schema.yaml
2025-04-10 17:57:58,398 - Datasciencelogger - INFO - created directory at: artifacts
2025-04-10 17:57:58,406 - Datasciencelogger - INFO - created directory at: artifacts/model_evaluation


2025-04-10 17:58:00,327 - Datasciencelogger - INFO - json file saved at: artifacts\model_evaluation\metrics.json


2025/04/10 17:58:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ElasticNetModel' already exists. Creating a new version of this model...
2025/04/10 17:58:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 2
Created version '2' of model 'ElasticNetModel'.


🏃 View run rogue-jay-457 at: https://dagshub.com/lassoinadame2018/Project1_Data_Science.mlflow/#/experiments/0/runs/688c2a63606145cdba586edc2ca45b1d
🧪 View experiment at: https://dagshub.com/lassoinadame2018/Project1_Data_Science.mlflow/#/experiments/0
